<a href="https://colab.research.google.com/github/maxigaarp/Gestion-De-Datos-en-R/blob/main/Taller_3_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 3

In [ ]:
install.packages("RSQLite")
install.packages("data.table")

In [ ]:
library(RSQLite)
library(tidyverse)

## Calidad de datos y Modelo de entidad relación

Para esta parte del taller vamos a trabajar con la base de datos de la matricula de estudiantes de Educación superior del año 2020

Para ello, vamos a descargar la base de datos y, al mismo tiempo, vamos a descargar el esquema de registro de la tabla, que uno puede encontrar en el siguiente link

http://datos.mineduc.cl/dashboards/20195/descarga-bases-de-datos-matricula-en-educacion-superior/

In [ ]:
system("wget http://datos.mineduc.cl/datasets/193401-matricula-de-educacion-superior-2020.download/")
system("mv /content/index.html /content/matricula_ES_2020.zip")

In [ ]:
unzip("/content/matricula_ES_2020.zip")



In [ ]:
system("gdown --id 1dXpk0MP6eXcgzfsFT4fnwjchecMwrNmZ")


In [ ]:
system("unzip /content/20200706_Matrcula_Ed_Superior_2020_PUBL_MRUN.zip")

In [ ]:
matricula <- read.csv2("20200706_Matrícula_Ed_Superior_2020_PUBL_MRUN.csv")

Lo primero que haremos con este trabajo, será elaborar un modelo entidad-relación. Para ello, debemos investigar el esquema de registro y responder una serie de preguntas:



1.   ¿Qué entidades existen?
2.   ¿Cómo se relacionan estas entidades?
3.   ¿Cuáles son los atributos de cada entidad?
4.   ¿Cuál es la llave de cada entidad?



In [ ]:
matricula[]

In [ ]:
str(matricula)

Para construir nuestra base de datos, ahora queremos determinar la calidad de los mismos y, eliminar o corregir aquellos datos que presenten problemas. Para ello, debemos trabajar cada una de las dimensiones.

La primera dimensión que vamos a evaluar es completitud. Aquí, revisamos los datos que falten en cada dimensión, observando cuantos datos faltan en cada columna. 

In [ ]:
df <-apply(X = is.na(matricula), MARGIN = 2, FUN = mean)
print(df*100)

¿Qué una columna presente una cantidad importante de datos faltantes, implica que hay un problema con estos datos?

¿Que datos debiesemos eliminar si o si?

Evaluada la completitud de los atributos, es necesario evaluar la validez de los mismos. Es decir, debemos evaluar si los datos cumplen con sus reglas y definiciones acordes.

Existen definiciones básicas de tipo de datos y definiciones más complejas que se refieren a los valores validos definidos por el creador de la base de datos.


In [ ]:
# Definiciones tipos de datos
# Año número entero
falla <- 0
for(c in matricula$cat_periodo){
#  print(c)
  if( typeof(c) != "integer" ){
   falla <- falla +1
  }
}
proporcion <- (falla / length(matricula$cat_periodo)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))


In [ ]:
# Nombre carrera character
falla <-0
for(c in matricula$nomb_carrera){

  if( typeof(c) != "character" ){
    falla <- falla +1
  }
  
}
proporcion = (falla / length(matricula$nomb_carrera)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))


**Actividad:** seleccione otros 3 atributos y revise su validez. Tiene 10 minutos

In [ ]:
# Definiciones de acuerdo al creador de la base de datos

# Genero entre 1 y 2
falla <-0
for(c in matricula$GEN_ALU){
  if(c != 1 & c != 2){
    falla <- falla +1
  }  
}

proporcion = (falla / length(matricula$GEN_ALU)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))



Una forma más directa y mas informativa tambien seria agrupar por GEN_ALU y contar cada tipo.

In [ ]:
matricula %>% group_by(GEN_ALU) %>%
              summarise(n())

In [ ]:
# Nivel global institución en categoría

falla <-0
for(c in matricula$nivel_global){
  if(c != "Pregrado" & c != "Postgrado" & c != "Postítulo"){
    falla <- falla +1
  }  
}

proporcion = (falla / length(matricula$nivel_global)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))

Tambien podemos contar los grupos para este caso.

In [ ]:
matricula %>% group_by(nivel_global) %>%
              summarise(n())

In [ ]:
# Nacimiento tiene sentido

falla <-0
for(c in matricula$FEC_NAC_ALU){
  anio <- c %/% 100
  mes <- c %% 100
  if( !is.na(anio)){
  if(anio < 1920 | anio > 2005 | mes == 0 | mes > 13){
    falla <- falla +1
  }  
  }
}

proporcion = (falla / length(matricula$FEC_NAC_ALU)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))



In [ ]:
datanacimiento  <- matricula %>%
          select("FEC_NAC_ALU") %>%
          mutate(NAC_ALU_AGNO= FEC_NAC_ALU %/% 100,
                      NAC_ALU_MES= FEC_NAC_ALU %% 100)
datanacimiento

In [ ]:
datanacimiento %>% summary()

Notese el año de nacimiento 2019

In [ ]:
datanacimiento %>% ggplot(aes(x = NAC_ALU_AGNO)) +
                geom_histogram(aes())


In [ ]:
datanacimiento %>% ggplot(aes(x = NAC_ALU_MES)) +
                geom_histogram(aes())


Busquemos los valores malos


In [ ]:
matricula %>% filter(FEC_NAC_ALU>200601)

**Actividad**: Lea en el esquema de registro diferentes reglas de validez determinadas para estos datos, y revise la validez de dos de ellos. 

Otra evaluación importante es de consistencia y realizar las ediciones. Para ello, revisaremos la información siguiendo lo dicho por el esquema de registro.




In [ ]:
# Duracion total carrera mayor que la duración estudio de la carera
falla <-0
i <- 1
largo <- length(matricula$dur_total_carr) 
while(i < largo){
  if(matricula$dur_estudio_carr[i] > matricula$dur_total_carr[i]){
    falla <- falla +1    
    }
  i <- i + 1
}

proporcion <- (falla / length(matricula$dur_total_carr)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))

In [ ]:
# Tipo institución 3 calza con Tipo institución 1

falla <- 0
i <- 1
largo <- length(matricula$tipo_inst_3) 
while(i < largo){
  if(matricula$tipo_inst_1[i] == "Centros de Formación Técnica"){
    if(matricula$tipo_inst_3[i] != "Centros de Formación Técnica" & matricula$tipo_inst_3[i] != "Centros de Formación Técnica Estatales" ){
     falla <- falla +1   
    }
  }
  if(matricula$tipo_inst_1[i] == "Institutos Profesionales"){
    if(matricula$tipo_inst_3[i] != "Institutos Profesionales"){
     falla <- falla +1         
    }
    }
  if(matricula$tipo_inst_1[i] == "Universidades"){
    if(matricula$tipo_inst_3[i] == "Institutos Profesionales" | matricula$tipo_inst_3[i] == "Centros de Formación Técnica" | matricula$tipo_inst_3[i] == "Centros de Formación Técnica Estatales" ){
     falla <- falla +1         
    }
    }

  i <- i + 1
}

proporcion <- (falla / length(matricula$tipo_inst_3)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))


In [ ]:
matricula %>% group_by(tipo_inst_1,tipo_inst_3) %>%
            summarise(n())

In [ ]:
# El rango de edad tiene que calzar con la fecha de nacimiento
falla <- 0
i <- 1
largo <- length(matricula$rango_edad) 
while(i < largo){
  anio <- matricula$FEC_NAC_ALU[i] %/% 100
  edad <- 2020 - anio
  if(matricula$rango_edad[i] == "Entre 15 y 19 años"){
    if(edad > 19 | edad < 15){
      falla <- falla + 1
    }
  }
  if(matricula$rango_edad[i] == "Entre 20 y 24 años"){
    if(edad > 24 | edad < 20){
      falla <- falla + 1
    }
  }
  if(matricula$rango_edad[i] == "Entre 25 y 29 años"){
    if(edad > 29 | edad < 25){
      falla <- falla + 1
    }
  }
  if(matricula$rango_edad[i] == "Entre 30 y 34 años"){
    if(edad > 34 | edad < 30){
      falla <- falla + 1
    }
  }
  if(matricula$rango_edad[i] == "Entre 35 y 39 años"){
    if(edad > 39 | edad < 35){
      falla <- falla + 1
    }
  }
  if(matricula$rango_edad[i] == "40 y más años"){
    if(edad < 39){
      falla <- falla + 1
    }
  }
  if(matricula$rango_edad[i] == "Sin Información"){
    if(!is.na(edad) & edad < 100){
      falla <- falla + 1
    }
 
  }
  i <- i + 1
}

proporcion <- (falla / length(matricula$rango_edad)) * 100
print(paste0("la tasa de error es de ", proporcion, "%"))


In [ ]:
datanacimiento  <- matricula %>%
          select("rango_edad", "FEC_NAC_ALU") %>%
          mutate(NAC_ALU_AGNO= FEC_NAC_ALU %/% 100,
                      NAC_ALU_MES= FEC_NAC_ALU %% 100)
datanacimiento

In [ ]:
datanacimiento %>% group_by(rango_edad) %>%
                  summarise(minedad=2020-max(NAC_ALU_AGNO), 
                            maxedad=2020-min(NAC_ALU_AGNO))

**Actividad:** Encuentre otros dos atributos en que sea necesario realizar ediciones y obtenga la cantidad de datos que presentan errores

## Consultas de bases de datos en R y SQL

Ahora, trabajaremos con la base de datos, almacenando los datos de forma más eficiente y realizando consultas apropiadas, como se vio en clases




In [ ]:
install.packages("data.table")
library(data.table)
install.packages(c("httr", "jsonlite"))
library(httr)
library(jsonlite)
install.packages("tictoc")
library("tictoc")
library(tidyverse)
install.packages("RSQLite")
library(RSQLite)
library(ggplot2)

In [ ]:
names(matricula)

In [ ]:

alumnos <- matricula[, c("MRUN", "GEN_ALU", "FEC_NAC_ALU")]
estudia_en <- matricula[,c("MRUN", "cod_inst", "cat_periodo", "rango_edad")] 
cursa <- matricula[,c("MRUN","codigo_unico", "cat_periodo", "rango_edad", "anio_ing_carr_ori", "sem_ing_carr_ori", "anio_ing_carr_act", "sem_ing_carr_act", "forma_ingreso")] 
carrera <- matricula[, c("codigo_unico", "cod_sede", "nomb_sede", "cod_carrera", "nomb_carrera", "modalidad", "jornada", "version", "tipo_plan_carr", "dur_estudio_carr", "dur_proceso_tit", "dur_total_carr", "region_sede", "provincia_sede", "comuna_sede", "nivel_global", "nivel_carrera_1", "nivel_carrera_2", "requisito_ingreso",  "vigencia_carrera", "valor_matricula", "valor_arancel", "codigo_demre", "area_conocimiento", "oecd_area",  "oecd_subarea", "area_carrera_generica", "acreditada_carr", "costo_proceso_titulacion",  "costo_obtencion_titulo_diploma" )]
IES <- matricula[, c("cod_inst","tipo_inst_1", "tipo_inst_2", "tipo_inst_3", "nomb_inst", "acreditada_inst", "acre_inst_desde_hasta", "acre_inst_anio")]
pertenece_a <- matricula[,c( "codigo_unico","cod_inst")]


In [ ]:
# Limpiar eliminando llaves vacíos

# Solo me interesa limpiar MRUN, ¿Por qué?

alumnos <- alumnos[!is.na(alumnos$MRUN), ]
cursa <- cursa[!is.na(cursa$MRUN), ]
estudia_en <- estudia_en[!is.na(estudia_en$MRUN), ]

In [ ]:
#Eliminamos información repetida
alumnos <- unique(alumnos)
carrera <- unique(carrera)
IES <- unique(IES)



In [ ]:
estudia_en <- unique(estudia_en)
cursa <- unique(cursa)
pertenece_a <- unique(pertenece_a)

In [ ]:
cursa

En primer lugar, generaremos una serie de consultas, para tratar de entender la tabla que tenemos

In [ ]:
conn <- dbConnect(RSQLite::SQLite(), "mineduc.db")

In [ ]:
dbWriteTable(conn , name = "IES", 
                  value = IES,
                  row.names = FALSE, overwrite = TRUE )

dbWriteTable(conn , name = "alumnos", 
                  value = alumnos,
                  row.names = FALSE, overwrite = TRUE )
dbWriteTable(conn , name = "carrera", 
                  value = carrera,
                  row.names = FALSE, overwrite = TRUE)

dbWriteTable(conn , name = "estudia_en", 
                  value = estudia_en,
                  row.names = FALSE, overwrite = TRUE)
dbWriteTable(conn , name = "pertenece_a", 
                  value = pertenece_a,
                  row.names = FALSE, overwrite = TRUE)

dbWriteTable(conn , name = "cursa", 
                  value = cursa,
                  row.names = FALSE, overwrite = TRUE)


In [ ]:
# Encuentre el código de institución de la Pontificia Universidad Católica de Chile
dbGetQuery(conn, '
select 
cod_inst , nomb_inst
from IES 
where nomb_inst == "PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE";')


In [ ]:
# Genere una lista con todas las carreras diferentes, y sus nombres, que pertenecen al área OCDE Educación
dbGetQuery(conn, '
select 
codigo_unico, nomb_carrera
from carrera 
where oecd_area == "Educación";')


In [ ]:
# Determine todas las sedes que tiene la Universidad Católica
dbGetQuery(conn, '
select 
distinct nomb_sede
from carrera, pertenece_a 
where carrera.codigo_unico = pertenece_a.codigo_unico
AND pertenece_a.cod_inst = 86;')


In [ ]:
# Genere una tabla con la cantidad personas pertenecen a cada sede de la Universidad Católica
dbGetQuery(conn, '
select 
nomb_sede, count(cursa.MRUN) as "Alumnos"
from carrera, cursa, pertenece_a 
where  cursa.codigo_unico = carrera.codigo_unico
and carrera.codigo_unico = pertenece_a.codigo_unico
and pertenece_a.cod_inst = 86
group by nomb_sede;')


In [ ]:
# Genere una tabla de la cantidad de sedes por institución y grafiquela
sedes <- dbGetQuery(conn, '
select 
nomb_inst, count(carrera.cod_sede) as "Cantidad_Sedes"
from carrera, IES, pertenece_a 
where  pertenece_a.codigo_unico = carrera.codigo_unico
and pertenece_a.cod_inst = IES.cod_inst
group by carrera.nomb_sede;')

sedes %>%  ggplot(aes(x=nomb_inst,y=Cantidad_Sedes)) + 
            geom_bar(stat="identity", position="stack")


In [ ]:
# Grafique que regiones tienen mayor cantidad de alumnos
sedes <- dbGetQuery(conn, '
select 
region_sede, count(cursa.MRUN) as "Alumnos_Region"
from carrera, cursa, pertenece_a 
where  cursa.codigo_unico = carrera.codigo_unico
and carrera.codigo_unico = pertenece_a.codigo_unico
group by carrera.region_sede;')

sedes %>%  ggplot(aes(x=region_sede,y=Alumnos_Region)) + 
            geom_bar(stat="identity", position="stack")